In [13]:
from enum import Enum
import pandas as pd
from scipy.stats import ttest_ind
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
train_inputs = []
train_outputs = []
test_inputs = []
test_outputs = []

#Enum for all the categories and their column number
class Col(Enum):
    Gender = 1
    Menstrual = 2 
    SleepPrior = 3 
    Prescription = 4 
    OralAntibiotics = 5 
    TopAntibiotics = 6 
    Eyes = 7 
    Hands = 8 
    Race = 9 
    Feeling = 10 
    Symptoms = 11 
    Facial = 12 
    LastVomit = 13 
    EnvAllergies = 14 
    Snore = 15 
    GrindTeeth = 16 
    EarInfect = 17 
    AllergyTime = 18 
    Skin = 19 
    OralSurgery = 20 
    HeartDisease = 21 
    BadTeeth = 22 
    EatPrior = 23 
    DrinkPrior = 24 
    GumDay = 25 
    YogurtWeek = 26 
    TeaWeek = 27 
    CoffeeWeek = 28 
    SodaWeek = 29 
    FastFoodWeek = 30 
    EatOutWeek = 31 
    Straw = 32 
    AddedSugar = 33 
    Spicy = 34 
    ShareDrink = 35 
    MeatWeek = 36 
    FreshWeek = 37 
    BrushTongue = 38 
    BrushTeethWeek = 39 
    TobaccoWeek = 40 
    FlossWeek = 41 
    DentistYear = 42 
    Toothbrush = 43 
    Mouthwash = 44 
    Toothpaste = 45 
    WashHandsEat = 46 
    WashHandsRestroom = 47 
    ChangeToothbrush = 48 
    BiteNails = 49 
    OrthoDevice = 50 
    Device = 51 
    Stress = 52 
    HandleStress = 53 
    Organized = 54 
    RateStress = 55 
    AvgSleep = 56 
    AvgWork = 57 
    Setting = 58 
    Environment = 59 
    UpDown = 60 
    Weight = 61 
    ExerciseMonth = 62 
    Transportation = 63 
    Orientation = 64 
    NearDogsWeek = 65 
    NearCatsWeek = 66 
    ShareBathroom = 67 
    BooksYear = 68 
    Alarms = 69 
    Patience = 70 
    Lipstick = 71 
    IntroExtro = 72 
    Relgious = 73 
    OptiPessi = 74 
    SexuallyActive = 75 
    OralSex = 76 
    Music = 77 
    AvgGrade = 78 
    RateHappy = 79 
    Arts = 80 
    Kiss = 81 
    Roommates = 82 

    
#Enum for type of test
class Test(Enum):
    Ttest = 1
    Anova = 2
    

def sleepPrior(category, data):
    if category != Col.SleepPrior:
        return data
    
    #if SleepPrior, manipulate the data so that the subcategories don't go over 7
    #also create new subcategories to make it more general
    data[category.name] = data[category.name].replace(['0', '1', '2', '3'], 'less than 4')
    data[category.name] = data[category.name].replace(['4', '5', '6', '7'], '4 to 7')
    data[category.name] = data[category.name].replace(['8', '9', '10', '11', '12'], 'more than 7')
    return data


def prescription(category, data):
    if category != Col.Prescription:
        return data
    
    #manipulate the data so that if the prescription column has a specific prescription, replace with yes
    data[category.name] = data[category.name].replace(['None', 'none', 'no '], 'no')
    data.loc[(data[category.name] != 'no') & (data[category.name] != 'No') & (data[category.name] != 'nan'), 
             category.name] = 'yes'
    return data


def eyes(category, data):
    if category != Col.Eyes:
        return data
    
    #manipulate the data to make it more generic
    data[category.name] = data[category.name].replace('Green/blue/gold mix', 'hazel')
    return data


def perWeek(category, data):
    if category not in (Col.TeaWeek, Col.CoffeeWeek, Col.SodaWeek, Col.FastFoodWeek, Col.BrushTeethWeek, 
                        Col.FlossWeek, Col.BooksYear, Col.GumDay, Col.EatOutWeek, Col.AvgSleep, Col.AvgWork, 
                        Col.ExerciseMonth) :
        return data
    
    #manipulate the data so results don't show as dates
    data[category.name] = data[category.name].replace('5-Jan', '1-5')
    data[category.name] = data[category.name].replace('10-Jun', '6-10')
    data[category.name] = data[category.name].replace('15-Nov', '11-15')
    
    data[category.name] = data[category.name].replace('2-Jan', '1-2')
    data[category.name] = data[category.name].replace('4-Mar', '3-4')
    data[category.name] = data[category.name].replace('6-May', '5-6')
    data[category.name] = data[category.name].replace('8-Jul', '7-8')
    data[category.name] = data[category.name].replace('10-Sep', '9-10')
    data[category.name] = data[category.name].replace('12-Nov', '11-12')
    
    if category == Col.FlossWeek:
        data[category.name] = data[category.name].replace('5-Nov', '11-15')
    
    return data


def numToWords(category, data):
    if category not in (Col.AddedSugar, Col.Straw, Col.Lipstick):
        return data
    
    #manipulate the data so results don't show as dates
    data[category.name] = data[category.name].replace('1', 'never')
    data[category.name] = data[category.name].replace('2', 'rarely')
    data[category.name] = data[category.name].replace('3', 'sometimes')
    data[category.name] = data[category.name].replace('4', 'often')
    data[category.name] = data[category.name].replace('5', 'always')
    
    return data


def mouthwash(category, data):
    if category != Col.Mouthwash:
        return data
    
    #manipulate the data so that results are in form of yes or no
    data.loc[(data[category.name] != 'never') & (data[category.name] != 'No') & (data[category.name] != 'nan'), 
             category.name] = 'yes'
    data[category.name] = data[category.name].replace('never', 'no')
    return data


def religious(category, data):
    if category != Col.Relgious:
        return data
    
    #manipulate the data so that results are in form of yes or no
    data.loc[(data[category.name] != 'No') & (data[category.name] != 'nan'), category.name] = 'yes'
    data[category.name] = data[category.name].replace('never', 'no')
    return data


def orientation(category, data):
    if category != Col.Orientation:
        return data
    
    #fix spelling mistakes, change bicurious to bisexual to generalize data
    data[category.name] = data[category.name].replace('hetrosexual', 'heterosexual')
    data[category.name] = data[category.name].replace(['bicurious', 'Bicurious'], 'bisexual')
    return data


bactClassDict = {}
    
def runTests(data, category, outputMap, showAllClasses):
    subcategory = []
    categoryDF = []
    bactClasses = []
    sigClasses = []
    inputs = []
    
    #decided not to do these groups
    if category.name in ('Menstrual', 'Race', 'Feeling', 'Symptoms', 'EnvAllergies', 'Skin', 'EatPrior',
                        'DrinkPrior', 'TobaccoWeek', 'Toothpaste', 'BiteNails', 'Transportation', 'Environment', 
                        'NearDogsWeek', 'NearCatsWeek'):
        return False
    
    
    # fix the inconsistant data
    data = sleepPrior(category, data)
    data = prescription(category, data)
    data = eyes(category, data)
    data = perWeek(category, data)
    data = numToWords(category, data)
    data = mouthwash(category, data)
    data = religious(category, data)
    data = orientation(category, data)

    #change the category into its own separate dataframe

    #get list of subcategories
    try:
        data[category.name] = data[category.name].str.lower()
    except:
        data[category.name]
    subcategory = data[category.name].drop_duplicates().values.tolist()
    
    #remove nans (where people didn't put in an answer)
    try:
        subcategory = [subcat.lower().strip() for subcat in subcategory if str(subcat) != 'nan' 
                   and str(subcat) != 'don\'t know' and str(subcat) != 'do not know' and str(subcat) != 'don?t know']
    except:
        subcategory = [subcat for subcat in subcategory if str(subcat) != 'nan' 
                   and str(subcat) != 'don\'t know' and str(subcat) != 'do not know' and str(subcat) != 'don?t know']
        
    #remove any more duplicates
    subcategory = list(set(subcategory))

    #remove outliers/format the data for specific categories
    if category == Col.Spicy:
        subcategory.remove('sometimes')
    if category == Col.Hands:
        subcategory.remove('ambidextrous')
    if category == Col.BrushTeethWeek:
        subcategory.remove('42689');
    

    #decide on which test to use
    if len(subcategory) == 2:
        test = Test.Ttest
    else:
        test = Test.Anova
        
    #load dataframes for each subcategory of all bacterial classes\n",
    for x in range(0, len(subcategory)):
        categoryDF.append(data[data[category.name] == subcategory[x]].iloc[:, 83:125])
        
    print(category.name)
    print(subcategory)
        
    #go through all bacterial classes to perform tests on them
    sigClassCount = 0
    for x in range(0, 42):
        
        #perform t-test if applicable
        if test == Test.Ttest:
            ttest = ttest_ind(categoryDF[0].iloc[:, x],categoryDF[1].iloc[:, x])
            pvalue = ttest[1]
            
        #perform anova test is applicable
        if test == Test.Anova:
            #keep track of how many subcategories there are to put into anova test
            arg = []
            for y in range(0, len(subcategory)):
                arg.append(categoryDF[y].iloc[:, x])
            anova = stats.f_oneway(*(a for a in arg))
            pvalue = anova[1]

        #keep a list of all bacteria classes and their index
        if len(bactClasses) != 42:
            bactClasses.append(categoryDF[0].iloc[:,x].name)

        #print classes that are significant
        if pvalue <= 0.05 or showAllClasses:
            #keep track of significant classes
            sigClassCount += 1
            className = categoryDF[0].iloc[:,x].name
            if className not in bactClassDict:
                bactClassDict[className] = 1
            else:
                bactClassDict[className] += 1
            sigClasses.append(className)

            print(categoryDF[0].iloc[:,x].name, "\nP-value: {:4.6f}".format(pvalue))

    # print out info for categories with high numbers of significant classes  
#     if sigClassCount >= 4: 
#         catData = []
#         print(sigClassCount)
#         print(sigClasses)
#         for subcat in data[category.name]:
#             if subcat in subcategory:
#                 catData.append(subcat)
#         print(len(catData))
#         print(catData)
    
    print('\n')
        
    #get and set inputs, key = bactClass, value = list of all values under that class - Design 1
#     for c in sigClasses:
#         classValue = []
#         for index, row in data.iterrows():
#             if row[category.name] in subcategory:
#                 classValue.append(row[c])
#         inputs[c] = classValue
#     print(inputs)


    # UNCOMMENT BELOW HERE FOR NN TESTING

    #get and set inputs Design 2
    for index, row in data.iterrows():
        classValue = []
        for c in sigClasses:
            if row[category.name] in subcategory:
                classValue.append(row[c])
        if row[category.name] in subcategory:
            inputs.append(classValue)
    print(len(inputs), inputs)
    
    #add subcategories to outputMap
    outputMap[category.name] = subcategory
#     print(outputMap)
    
    #get and set outputs so that it looks like [a,b,c]
    outputs = []
    for index, row in data.iterrows():
        if row[category.name] in subcategory:
            outputs.append(row[category.name])
#     print(len(outputs), outputs)
    
    #replace string outputs with numbers
    if outputs and isinstance(outputs[0], str):
        for i in range(0, len(outputs)):
            outputs[i] = subcategory.index(outputs[i])
#     print(len(outputs), outputs)
    
    #separate training and testing data
    for i in range(0, len(inputs)):
        if i % 2 == 1:
            train_inputs.append(inputs[i])
            train_outputs.append(outputs[i])
        else:
            test_inputs.append(inputs[i])
            test_outputs.append(outputs[i])  
    
    

class Main():
    data = pd.read_csv('MasterSheet.csv')
    outputMap = {}
    showAllClasses = False
    
    runTests(data, Col.Gender, outputMap, showAllClasses)
    
#     for x in range(1, 82):
#         runTests(data, Col(x), outputMap, showAllClasses)
        
    #print out the highest appearing sig class
#     sort = sorted(bactClassDict.items(), key=lambda x: x[1], reverse=True)
#     print(sort)



Gender
['male', 'female']
Bacilli 
P-value: 0.040059
Fusobacteria 
P-value: 0.000003
Epsilonproteobacteria 
P-value: 0.021050
Alphaproteobacteria 
P-value: 0.038678
Thermotogae 
P-value: 0.012158


106 [[3.9442973119999998, 0.076904877, -0.33510018199999997, -0.342277971, -0.353762432], [3.9723758339999997, 0.190906131, -0.362045826, -0.357471775, -0.381358487], [3.760585735, 0.602311415, -0.370152097, -0.361497044, -0.376256186], [3.114230698, -0.22210792399999998, -0.339227998, -0.318056067, -0.331408263], [5.343307545, 0.134594481, -0.308472788, -0.30015785300000003, -0.319163419], [3.503269346, 0.480379028, -0.363317868, -0.366886806, -0.377593619], [3.746173966, -0.021308512, -0.35812048399999996, -0.329250886, -0.343685685], [2.187348146, -0.207054766, -0.33773129700000004, -0.328490128, -0.336431133], [3.77699665, 0.203021108, -0.31253190000000003, -0.35530704, -0.35530704], [4.293834467, -0.064573619, -0.349397739, -0.333604432, -0.351576126], [4.398598967, 0.094596186, -0.3224

In [27]:
# following https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/_index.ipynb
# as a tutorial

import tensorflow as tf
from tensorflow.keras import layers

mnist = tf.keras.datasets.mnist

# (x_train, y_train), (x_test, y_test) = mnist.load_data()
# x_train, x_test = x_train / 255.0, x_test / 255.0


# model = tf.keras.models.Sequential([
#   tf.keras.layers.Flatten(),
#   tf.keras.layers.Dense(512, activation=tf.nn.relu),
#   tf.keras.layers.Dropout(0.2),
#   tf.keras.layers.Dense(10, activation=tf.nn.softmax)
# ])
model = tf.keras.Sequential()
model.add(layers.Dense(units=4,input_shape=(5,)))
model.add(layers.Dense(units=4))
model.add(layers.Dense(units=2))
# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# model.fit(np.array(train_inputs), np.array(train_outputs), epochs=5000)
model.fit(np.array(train_inputs), np.array(train_outputs), epochs=800,
              validation_data=(np.array(test_inputs), np.array(test_outputs)))

# model.evaluate(np.array(test_inputs), np.array(test_outputs))

Train on 53 samples, validate on 53 samples
Epoch 1/800
53/53 [==============================] - 0s 7ms/step - loss: 4.5472 - acc: 0.3585 - val_loss: 1.9069 - val_acc: 0.5094
Epoch 2/800
53/53 [==============================] - 0s 113us/step - loss: 1.8948 - acc: 0.5283 - val_loss: 1.0433 - val_acc: 0.5094
Epoch 3/800
53/53 [==============================] - 0s 207us/step - loss: 1.5872 - acc: 0.5283 - val_loss: 1.0288 - val_acc: 0.5094
Epoch 4/800
53/53 [==============================] - 0s 113us/step - loss: 1.3518 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 5/800
53/53 [==============================] - 0s 132us/step - loss: 0.6971 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 6/800
53/53 [==============================] - 0s 151us/step - loss: 0.6970 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 7/800
53/53 [==============================] - 0s 169us/step - loss: 0.6968 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 8/800
53/53 [========

Epoch 62/800
53/53 [==============================] - 0s 169us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 63/800
53/53 [==============================] - 0s 151us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 64/800
53/53 [==============================] - 0s 113us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 65/800
53/53 [==============================] - 0s 132us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 66/800
53/53 [==============================] - 0s 132us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 67/800
53/53 [==============================] - 0s 132us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 68/800
53/53 [==============================] - 0s 113us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 69/800
53/53 [==============================] - 0s 226us

Epoch 122/800
53/53 [==============================] - 0s 169us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 123/800
53/53 [==============================] - 0s 169us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 124/800
53/53 [==============================] - 0s 151us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 125/800
53/53 [==============================] - 0s 151us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 126/800
53/53 [==============================] - 0s 132us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 127/800
53/53 [==============================] - ETA: 0s - loss: 0.6931 - acc: 0.468 - 0s 151us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 128/800
53/53 [==============================] - 0s 226us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 129/800
53/

Epoch 182/800
53/53 [==============================] - 0s 188us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 183/800
53/53 [==============================] - 0s 169us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 184/800
53/53 [==============================] - 0s 151us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 185/800
53/53 [==============================] - 0s 132us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 186/800
53/53 [==============================] - 0s 132us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 187/800
53/53 [==============================] - 0s 188us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 188/800
53/53 [==============================] - 0s 245us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 189/800
53/53 [==============================] - 

Epoch 242/800
53/53 [==============================] - ETA: 0s - loss: 0.6931 - acc: 0.593 - 0s 132us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 243/800
53/53 [==============================] - 0s 151us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 244/800
53/53 [==============================] - 0s 151us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 245/800
53/53 [==============================] - 0s 150us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 246/800
53/53 [==============================] - 0s 169us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 247/800
53/53 [==============================] - 0s 245us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 248/800
53/53 [==============================] - 0s 132us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 249/800
53/

...

Epoch 541/800
53/53 [==============================] - 0s 151us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 542/800
53/53 [==============================] - ETA: 0s - loss: 0.6931 - acc: 0.562 - 0s 151us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 543/800
53/53 [==============================] - 0s 169us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 544/800
53/53 [==============================] - 0s 151us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 545/800
53/53 [==============================] - 0s 151us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 546/800
53/53 [==============================] - 0s 169us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 547/800
53/53 [==============================] - 0s 150us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 548/800
53/

53/53 [==============================] - 0s 151us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 602/800
53/53 [==============================] - 0s 320us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 603/800
53/53 [==============================] - ETA: 0s - loss: 0.6931 - acc: 0.562 - 0s 169us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 604/800
53/53 [==============================] - 0s 132us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 605/800
53/53 [==============================] - 0s 169us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 606/800
53/53 [==============================] - 0s 132us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 607/800
53/53 [==============================] - 0s 132us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 608/800
53/53 [==========

Epoch 661/800
53/53 [==============================] - 0s 132us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 662/800
53/53 [==============================] - 0s 169us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 663/800
53/53 [==============================] - 0s 151us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 664/800
53/53 [==============================] - 0s 151us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 665/800
53/53 [==============================] - 0s 263us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 666/800
53/53 [==============================] - 0s 169us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 667/800
53/53 [==============================] - 0s 169us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 668/800
53/53 [==============================] - 

53/53 [==============================] - 0s 169us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 722/800
53/53 [==============================] - 0s 188us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 723/800
53/53 [==============================] - 0s 245us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 724/800
53/53 [==============================] - 0s 169us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 725/800
53/53 [==============================] - 0s 151us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 726/800
53/53 [==============================] - 0s 188us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 727/800
53/53 [==============================] - 0s 188us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 728/800
53/53 [==============================] - 0s 151us/step 

53/53 [==============================] - 0s 132us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 782/800
53/53 [==============================] - 0s 151us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 783/800
53/53 [==============================] - 0s 132us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 784/800
53/53 [==============================] - 0s 132us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 785/800
53/53 [==============================] - 0s 263us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 786/800
53/53 [==============================] - 0s 132us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 787/800
53/53 [==============================] - 0s 151us/step - loss: 0.6931 - acc: 0.5283 - val_loss: 0.6931 - val_acc: 0.5094
Epoch 788/800
53/53 [==============================] - 0s 132us/step 